In [1]:
import numpy as np 
import pandas as pd
import os
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

In [2]:
path = Path.cwd().parent / 'data'
for dirname, _, filenames in os.walk(path):
    for filename in filenames:
        print(os.path.join(filename))

gender_submission.csv
test.csv
train.csv


In [3]:
df = pd.read_csv(path / 'train.csv')
df_test = pd.read_csv(path / 'test.csv')
df.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [4]:
df.dropna(subset=['Age'], inplace=True)
df_test = df_test.fillna(0)

In [5]:
y_train_data = df['Survived']
features = df.drop(columns=['Cabin', 'Embarked', 'Ticket', 'Name', 'Survived'])
features_test = df_test.drop(columns=['Cabin', 'Embarked', 'Ticket', 'Name'])

In [6]:
X_train_data = pd.get_dummies(features)
X_test = pd.get_dummies(features_test)

In [7]:
X_train, X_val, y_train, y_val = train_test_split(X_train_data, y_train_data, test_size=0.2,random_state=42)

In [8]:
random_forest = RandomForestClassifier(random_state=42)
parameters = {'n_estimators': [100, 150, 200, 250, 300, 350, 400, 450, 500],
              'max_depth': [1, 2 , 5, 6, 7, 8, 9],
             'min_samples_split': [2,20,200]}
clf = GridSearchCV(random_forest, parameters, cv=3, n_jobs=-1)
clf.fit(X_train, y_train)
sorted(clf.cv_results_.keys())
clf.best_params_

{'max_depth': 8, 'min_samples_split': 2, 'n_estimators': 200}

In [9]:
dat_frame = pd.DataFrame(clf.cv_results_)

dat_frame.sort_values('mean_test_score',ascending=False)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_min_samples_split,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
137,0.663588,0.027023,0.038868,0.007001,8,2,200,"{'max_depth': 8, 'min_samples_split': 2, 'n_es...",0.811518,0.831579,0.847368,0.830155,0.014670,1
170,1.874004,0.086831,0.081377,0.003550,9,2,500,"{'max_depth': 9, 'min_samples_split': 2, 'n_es...",0.821990,0.847368,0.821053,0.830137,0.012191,2
138,0.829816,0.047780,0.037367,0.000570,8,2,250,"{'max_depth': 8, 'min_samples_split': 2, 'n_es...",0.811518,0.826316,0.847368,0.828401,0.014710,3
168,1.409829,0.177981,0.061222,0.003099,9,2,400,"{'max_depth': 9, 'min_samples_split': 2, 'n_es...",0.821990,0.842105,0.821053,0.828382,0.009711,4
169,1.505306,0.116930,0.087364,0.016279,9,2,450,"{'max_depth': 9, 'min_samples_split': 2, 'n_es...",0.821990,0.842105,0.821053,0.828382,0.009711,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,0.407037,0.081259,0.032485,0.015831,5,200,100,"{'max_depth': 5, 'min_samples_split': 200, 'n_...",0.774869,0.784211,0.773684,0.777588,0.004708,184
153,0.300256,0.029965,0.018962,0.002767,8,200,100,"{'max_depth': 8, 'min_samples_split': 200, 'n_...",0.774869,0.784211,0.773684,0.777588,0.004708,184
126,0.358754,0.065276,0.017932,0.002058,7,200,100,"{'max_depth': 7, 'min_samples_split': 200, 'n_...",0.774869,0.784211,0.773684,0.777588,0.004708,184
180,0.325430,0.010469,0.020495,0.002278,9,200,100,"{'max_depth': 9, 'min_samples_split': 200, 'n_...",0.774869,0.784211,0.773684,0.777588,0.004708,184


In [10]:
output = pd.DataFrame({'PassengerId': df_test.PassengerId,'Survived': clf.predict(X_test)})

output.to_csv('submission.csv',index=0)